# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [2]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the artifact from Lab 1
# Note: load_artifact() automatically parses JSON files, so no need to call json.loads()
user_stories_data = load_artifact("day1_user_stories.json")
if not user_stories_data:
    user_stories_data = []

2025-10-27 17:12:17,974 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [3]:
# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f"""
You are acting as a product manager. I need a well formatted, clean markdown Product Requirements Document (PRD) with three sections: Introdcuction, User Personas, and Features/User Stories. This input to be used for this document is {user_stories_data}
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
Of course. Here is a well-formatted Product Requirements Document (PRD) based on the input you provided.

# Product Requirements Document: Employee Onboarding Platform

**Version:** 1.0
**Date:** October 26, 2023
**Author:** Product Manager
**Status:** Draft

---

## 1. Introduction

### 1.1 Problem Statement
Companies struggle to provide a consistent, engaging, and efficient onboarding experience for new employees. This leads to new hire anxiety and confusion, significant administrative overhead for managers and HR, and potential compliance risks. Without a structured process, new hires take longer to become productive, and their long-term engagement and retention can be negatively impacted.

### 1.2 Vision & Goals
Our vision is to create a seamless, engaging, and scalable onboarding experience that empowers new hires to feel welcome and productive from day one.

The primary goals of this platform are to:
*   **Improve New Hire Satisfaction:** Reduce firs

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [5]:
# Load the PRD template
# Note: templates directory is at project root, not in artifacts, so we need to specify the base_dir
from utils import detect_project_root
prd_template_content = load_artifact("templates/prd_template.md", base_dir=detect_project_root())

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""You are a Senior Product Manager with 10+ years of experience creating comprehensive Product Requirements Documents (PRDs) for enterprise software products. You excel at synthesizing user stories into clear, actionable product documentation that aligns stakeholders and guides development teams.

CONTEXT:
You have been provided with a set of validated user stories for an onboarding tool designed to help companies get new hires up to speed more efficiently. Your task is to create a complete, professional Product Requirements Document based on these user stories.

USER STORIES DATA:
{json.dumps(user_stories_data, indent=2)}

PRD TEMPLATE STRUCTURE:
{prd_template_content}

INSTRUCTIONS:
1. Use the PRD template structure provided above as your exact framework. Do not omit any sections.
2. Populate EVERY section of the template with relevant, professional content.
3. For Section 4 (Functional Requirements & User Stories), organize the user stories by logical feature groups or epics. Present each user story with its full acceptance criteria in Gherkin format.
4. For sections NOT directly covered in the user stories (such as "Goals & Success Metrics", "Non-Functional Requirements", "Out of Scope", etc.), use your expertise to infer and create logical, realistic content based on:
   - The overall goal of streamlining new hire onboarding
   - Common industry best practices for onboarding systems
   - Typical enterprise software requirements
5. Ensure the Executive Summary captures the essence of the entire product vision.
6. In the "Goals & Success Metrics" section, create a realistic table with specific KPIs and measurable targets.
7. For "Out of Scope", identify features that would be valuable but are intentionally excluded from the initial release.
8. Replace all placeholder text (e.g., [Product Name], [Your Name], [Date]) with appropriate values.
9. Maintain a professional, clear, and actionable tone throughout.

OUTPUT REQUIREMENTS:
- Your response must be valid markdown format
- Follow the exact section structure and numbering from the template
- Be thorough and comprehensive - this is a formal document for stakeholders
- Ensure consistency in terminology and formatting throughout

Generate the complete PRD now."""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

--- Generating PRD from Template ---
Of course. As a Senior Product Manager, I've synthesized the user stories into a comprehensive and actionable PRD. Here is the complete document.

***

# Product Requirements Document: Momentum Onboarding Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Senior Product Manager |
| **Version** | 1.0 |
| **Last Updated** | October 26, 2023 |

## 1. Executive Summary & Vision
Momentum is an integrated onboarding platform designed to streamline and standardize the new hire experience. We are building Momentum to solve the widespread problem of inconsistent, manual, and overwhelming onboarding processes that lead to decreased new hire productivity, increased administrative burden, and lower employee engagement. Our vision is to create a single source of truth for onboarding that empowers HR, equips managers, and excites new hires, ultimately accelerating time-to-contribution and fostering a stronger sense of belonging from day one.

## 2. T

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [ ]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f"""
# Your prompt here
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md")

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.